# Sistemas Baseados em Conhecimento
## Sistemas Baseados em Regras
### Pyknow

Fatos

 São uma unidade básica de informação de Pyknow
 
 A classe Fact é uma subclasse de dict (dicionário)
 
 Portanto, Fact não mantém uma ordem interna dos seus itens

In [1]:
from pyknow import *

f = Fact(a=1, b=2)
print(f['a'])

1


É possível criar fatos sem palavras chaves

In [2]:
f = Fact ('x', 'y', 'z', a = 1, b = 2)
print (f[0])
print(f['a'])

x
1


É possível criar subclasses de Fact para expressar tipos diferentes de dados e/ou criar funcionalidades específicas

In [3]:
class Alert(Fact):
    pass

class Status(Fact):
    pass

f1 = Alert(color = 'red')
f2 = Status (state = 'critical')
print (f1['color'])
print (f2['state'])

red
critical


Inicialização de fatos

 Sistema necessita para começar a funcionar de um conjunto de fatos iniciais.
 
 Todos os fatos iniciais definidos nos decoradores DefFacts são criados toda vez que o método reset é chamado.

In [4]:
@DefFacts()
def needed_data():
    yield Fact(best_color = 'red')
    yield Fact(best_body = 'medium')
    yield Fact(best_sweetness = 'dry')     

Regras

 Tem dois componentes (LHS e RHS). 
 
 LHS descrevem as condições para a regra ser disparada. 
 
 RHS é um conjunto de ações realizadas quando a regra é disparada

In [5]:
@Rule(AND(Fact(feature='voa'), 
          Fact(tipo='ave')))
def e_passaro(self):
    print("Passaro")

@Rule(AND(NOT(Fact(feature='voa')),
          Fact(tipo='ave')))
def e_pinguim(self):
    print("Pinguim")

Máquina de Inferência

 Responsável pela execução do sistema. 
 
 Primeiro passo é fazer uma subclasse. 
 
 Depois instanciá-la e rodar o sistema.

In [15]:
class ClassifyAnimals(KnowledgeEngine):

    @DefFacts()
    def _initial_features(self):
        yield Fact(feature="penas")
        yield Fact(feature="voa")

    @Rule(Fact(feature='penas'))
    def e_ave(self):
        self.declare(Fact(tipo='ave'))

    @Rule(AND(Fact(feature='voa'),
              Fact(tipo='ave')))
    def e_passaro(self):
        print("Passaro")

    @Rule(AND(NOT(Fact(feature='voa')),
              Fact(tipo='ave')))
    def e_pinguim(self):
        print("Pinguim")

engine = ClassifyAnimals()
engine.reset()
engine.run() 

Passaro


Operador existencial EXIST 

In [7]:
class Goal(Fact):
    pass

class Hero(Fact):
    name = Field(str)
    status = Field(str, default="unoccupied")

class Heroes(KnowledgeEngine):
    
        @DefFacts()
        def goal_and_heroes(self):
            yield Goal('save-the-day')
            yield Hero(name="Death Defying Man", status = 'busy')
            yield Hero(name="Stupendous Man")
            yield Hero(name="Incredible Man", status = 'busy')
            
        @Rule(Goal('save-the-day'),
              EXISTS(Hero(status="unoccupied")))
        def save_the_day(self):
            print("The day is saved")
            
engineH = Heroes()
engineH.reset()
engineH.run()


The day is saved


Wildcards: Associam com quaisquer valores.

MATCH: Retorna valor associado com nome.

In [16]:
class Greetings(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="greet")

    @Rule(Fact(action='greet'),
          NOT(Fact(name=W())))
    def ask_name(self):
        self.declare(Fact(name=input("What's your name? ")))

    @Rule(Fact(action='greet'),
          NOT(Fact(location=W())))
    def ask_location(self):
        self.declare(Fact(location=input("Where are you? ")))

    @Rule(Fact(action='greet'),
          Fact(name=MATCH.name),
          Fact(location=MATCH.location))
    def greet(self, name, location):
        print("Hi %s! How is the weather in %s?" % (name, location))

engine = Greetings()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!

What's your name? João
Where are you? São Paulo
Hi João! How is the weather in São Paulo?


L: Só associa exatamente com o valor literal usado. 

&: Associa se todos os valores casam.

|: Associa se um dos valores casam.

~: Associa se o valor não casa. 

<<: Associa valor de fato a variável. 

AS: Associa fato a variável.

In [23]:
from random import choice

class Light(Fact):
    """Info about the traffic light."""
    pass

class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])

engine = RobotCrossStreet()
engine.reset()
engine.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine.run() 

Be cautious because light is blinking-yellow


Exercícios de Fixação

1. Qual a diferença entre um sistema baseado em conhecimento e um sistema convencional que executa o mesmo tipo de tarefa? Porque sistemas baseados em conhecimento foram chamados inicialmente de sistemas especialistas?

2. Qual a forma mais comum de representação de conhecimento em sistemas baseados em conhecimento? Qual a principal vantagem desse tipo de representação?

3. Explique o que são os modos de inferência com encadeamento para frente e para trás? Apresente exemplos.

4. Quais são consideradas as maiores dificuldades para a criação e uso de Sistemas Baseados em Conhecimento? Explique porque essas dificuldades ocorrem? 

5. Uma das alegadas vantagens de Sistemas Baseados em Conhecimento é possuírem capacidade de explicar seu raciocínio. Porque esses sistemas tem essa capacidade? Contraste isso com sistemas de aprendizado baseados em redes neurais, que não possuem essa capacidade. Porque explicar o raciocínio é importante e o que se perde quando não se tem essa capacidade? 

6. Explique em que tipo de situação é indicado o uso de Sistemas Baseados em Conhecimento?

7. Altere a implementação da classe ClassifyAnimals para que a inferência conclua Penguim em vez de Pássaro? Altere a implementação novamente para que seja perguntado ao usuário se o animal voa. Altere mais uma vez a implementação para que seja perguntado se o animal tem penas. Em caso negativo, o sistema deve responder que o animal é um morcego.